# TP5: Clase 6

Utilizando la webcam, implementar un algoritmo de seguimiento con CamShift que:

a - Permita elegir la ROI del objeto a seguir

b - Permita cambiar la escala de la ventana (no necesariamente la orientación)


In [2]:
# Se importan las librerías
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
%matplotlib

Using matplotlib backend: Qt5Agg


In [66]:

def get_ROI(frame):
    fromCenter = False
    rec = cv.selectROI(frame, fromCenter)
    track_window = (rec[0], rec[1], rec[3], rec[2])
    roi = frame[int(rec[1]):int(rec[1] + rec[3]), int(rec[0]):int(rec[0] + rec[2])]
    return roi, track_window

def convert_HSV(im):
    return cv.cvtColor(im, cv.COLOR_BGR2HSV)

def plot_histograms(im):
    h, s, v = im[:,:,0], im[:,:,1], im[:,:,2]
    hist_h = cv.calcHist([h], [0], None, [256], [0,256])
    hist_s = cv.calcHist([s], [0], None, [256], [0,256])
    hist_v = cv.calcHist([v], [0], None, [256], [0,256])
    plt.plot(hist_h, color='r', label="H")
    plt.plot(hist_s, color='g', label="S")
    plt.plot(hist_v, color='b', label="V")
    plt.legend()
    plt.grid()
    plt.show()

def get_norm_hist(hsv_im, mask):
    hist = cv.calcHist([hsv_im], [0], mask, [180], [0, 180])
    cv.normalize(hist, hist, 0, 255, cv.NORM_MINMAX);
    return hist

def get_hsv_limits(im):
    def update_trackbar(val):
        pass

    window_name = "Mask selector"
    cv.namedWindow(window_name)
    bars = ["H_low", "H_high", "S_low", "S_high", "V_low", "V_high"]
    for bar_name in bars:
        value = 255 if "high" in bar_name else 0
        cv.createTrackbar(bar_name, window_name, value, 255, update_trackbar)
    hsv_limits = dict()
    while True:
        for bar_name in bars:
            hsv_limits[bar_name] = cv.getTrackbarPos(bar_name, window_name)
        
        low_hsv = np.array((hsv_limits["H_low"], hsv_limits["S_low"], hsv_limits["V_low"]))
        high_hsv = np.array((hsv_limits["H_high"], hsv_limits["S_high"], hsv_limits["V_high"]))
        mask = cv.inRange(frame, low_hsv, high_hsv)
        
        cv.imshow("frame", frame)
        cv.imshow("mask", mask)

        key = cv.waitKey(1)
        if key == 27 or k == 13:
            break
    return low_hsv, high_hsv

    

In [82]:
# Se define el número de cámara y se hace la captura del video
cv.destroyAllWindows()
camera_number = 1
camera = cv.VideoCapture(camera_number)

# Se eliminan los primeros frames
for _ in range(4):
    ret, frame = camera.read()

# Se obtiene la ROI
roi, track_window = get_ROI(frame)
cv.destroyAllWindows()
print(f"Track window: {track_window}")
cv.imshow("ROI", roi)

# Se pasa la ROI a HSV (más estables a cambios de intensidad)
hsv_roi = convert_HSV(roi)
plot_histograms(hsv_roi)

Track window: (300, 325, 34, 111)


In [84]:
# Se convierte a hsv
hsv_frame = convert_HSV(frame)
# Se obtienen los limites para la máscara
low_hsv, high_hsv = get_hsv_limits(hsv_frame)
# Se define una máscara para el histograma 
mask = cv.inRange(hsv_roi, low_hsv, high_hsv)
# Se calcula el histograma sobre esa máscara (toma solo el Hue: tonalidad)
roi_hist = get_norm_hist(hsv_roi, mask)
# Se obtiene el histograma
cv.destroyAllWindows()


In [85]:
# Condición de terminación: 10 iteraciones o moverse al menos 1pt
term_crit = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 1, 10)

while(1):
    ret, frame = camera.read()
    if ret == True:
        hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
        dst = cv.calcBackProject([hsv], [0], roi_hist, [0, 180] ,1)
        
        # Se aplica camshift al nuevo frame
        ret, track_window = cv.CamShift(dst, track_window, term_crit)
        
        # Se dibuja la ventana sobre la imagen
        x, y, w, h = track_window
        img_rec = cv.rectangle(frame, (x, y), (x + w, y + h), 255, 2)
        
        cv.imshow('Seguimiento', img_rec)
        
        k = cv.waitKey(30) & 0xff
        if k == 27:
            break
    else:
        break
cv.destroyAllWindows()
camera.release()